# Implementation of ChatBot using NLP

In [21]:
pip install nltk scikit-learn streamlit

Note: you may need to restart the kernel to use updated packages.


In [22]:
#importing ne
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [23]:
ssl._create_default_https_context=ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\muthy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
intents = [
    {
        'tag': 'greeting',
        'patterns': ['Hi', 'Hello', 'Hey', 'Whats up', 'How are you'],
        'responses': ['Hi there', 'Hello', 'Hey', 'Nothing much', 'I\'m fine, thank you']
    },
    {
        'tag': 'goodbye',
        'patterns': ['Bye', 'See you later', 'GoodBye', 'Take care'],
        'responses': ['Goodbye', 'See you later', 'Take care']
    },
    {
        'tag': 'thanks',
        'patterns': ['Thanks', 'Thank you', 'Thanks a lot', 'Much appreciated'],
        'responses': ['You\'re welcome!', 'No problem!', 'Glad to help!', 'Anytime!']
    },
    {
        'tag': 'about',
        'patterns': ['Tell me about yourself', 'Who are you?', 'What are you?'],
        'responses': ['I am a chatbot here to assist you!', 'I am an AI assistant.', 'I am here to help with your queries.']
    },
    {
        'tag': 'help',
        'patterns': ['Can you help me?', 'I need help', 'Help me please'],
        'responses': ['Of course! What do you need help with?', 'I\'m here to help.', 'Sure, let me know what you need assistance with.']
    },
    {
        'tag': 'age',
        'patterns': ['How old are you?', 'What is your age?', 'When were you created?'],
        'responses': ['I am timeless!', 'I was created to assist you anytime.', 'Age doesn\'t apply to me, but I\'m always here for you.']
    },
    {
        'tag': 'weather',
        'patterns': ['What\'s the weather like?', 'How is the weather today?', 'Tell me the weather'],
        'responses': ['I can check the weather for you! Where are you located?', 'Let me get the latest weather update for you.']
    },
    {
        'tag': 'budget',
        'patterns': ['How do I plan a budget?', 'Tips for budgeting', 'Help me with my budget'],
        'responses': ['A good budget starts with tracking income and expenses.', 'Make sure to set financial goals and stick to them.', 'I can give you budgeting tips based on your needs.']
    },
    {
        'tag': 'credit-score',
        'patterns': ['What is a credit score?', 'How to improve my credit score?', 'Tell me about credit scores'],
        'responses': ['A credit score is a number that represents your creditworthiness.', 'To improve your credit score, pay bills on time and reduce debt.', 'Your credit score depends on factors like payment history and credit usage.']
    },
    {
        'tag': 'joke',
        'patterns': ['Tell me a joke', 'Make me laugh', 'Give me a joke'],
        'responses': ['Why don\'t scientists trust atoms? Because they make up everything!', 'What do you call fake spaghetti? An impasta!', 'Why did the scarecrow win an award? Because he was outstanding in his field!']
    },
    {
        'tag': 'motivation',
        'patterns': ['Give me motivation', 'Motivate me', 'I need inspiration'],
        'responses': ['Believe in yourself and all that you are!', 'Every day is a new beginning, take a deep breath and start again.', 'Success is not final, failure is not fatal: it is the courage to continue that counts.']
    },
    {
        'tag': 'study-tips',
        'patterns': ['How to study effectively?', 'Give me study tips', 'Best way to study?'],
        'responses': ['Stay organized and create a study schedule.', 'Use active recall and spaced repetition for better retention.', 'Take short breaks between study sessions to stay focused.']
    },
    {
        'tag': 'time-management',
        'patterns': ['How to manage time?', 'Time management tips', 'Best way to manage time?'],
        'responses': ['Prioritize tasks and avoid distractions.', 'Use a planner or a to-do list to stay organized.', 'Break large tasks into smaller chunks and set deadlines.']
    },
    {
        'tag': 'exercise',
        'patterns': ['Best exercises?', 'How to stay fit?', 'Give me fitness tips'],
        'responses': ['Regular exercise like walking, running, or yoga keeps you healthy.', 'Stay consistent with your workouts for better results.', 'Balance your fitness routine with a healthy diet.']
    },
    {
        'tag': 'healthy-eating',
        'patterns': ['What should I eat to stay healthy?', 'Healthy eating tips', 'Best diet advice?'],
        'responses': ['Eat more fruits, vegetables, and whole grains.', 'Stay hydrated and avoid processed foods.', 'Maintain a balanced diet with proteins, carbs, and fats.']
    },
    {
        'tag': 'technology',
        'patterns': ['Tell me about AI', 'Latest tech trends', 'What’s new in technology?'],
        'responses': ['AI is transforming industries with automation and machine learning.', 'The latest tech trends include AI, blockchain, and IoT.', 'Quantum computing is advancing rapidly in the tech world.']
    },
    {
        'tag': 'travel',
        'patterns': ['Where should I travel?', 'Best travel destinations?', 'Give me travel advice'],
        'responses': ['It depends on your preferences! Do you like beaches, mountains, or cities?', 'Popular destinations include Paris, Bali, and Tokyo.', 'Always research your destination and plan accordingly.']
    },
    {
        'tag': 'movies',
        'patterns': ['Recommend me a movie', 'Best movies to watch?', 'Give me a movie suggestion'],
        'responses': ['If you like action, try \'Inception\' or \'Mad Max: Fury Road\'.', 'For comedy, check out \'Superbad\' or \'The Grand Budapest Hotel\'.', 'If you love sci-fi, watch \'Interstellar\' or \'Blade Runner 2049\'.']
    },
    {
        'tag': 'books',
        'patterns': ['Recommend me a book', 'Best books to read?', 'Give me a book suggestion'],
        'responses': ['For fiction, try \'To Kill a Mockingbird\' or \'1984\'.', 'If you like self-help, read \'Atomic Habits\' or \'The 7 Habits of Highly Effective People\'.', 'For fantasy, \'Harry Potter\' and \'The Lord of the Rings\' are classics.']
    }
]


In [25]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

#Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

In [27]:
#Training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x,y)

LogisticRegression(max_iter=10000, random_state=0)

In [29]:
# Python function to chat with the chatbot
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [31]:
# Checking our chatbot.
user_input = "What's your age?"
response = chatbot(user_input)
print(response)

I am timeless!


In [33]:
user_input = "Hello"
response = chatbot(user_input)
print(response)

I'm fine, thank you


In [34]:
user_input = "Tell me about AI"
response = chatbot(user_input)
print(response)

The latest tech trends include AI, blockchain, and IoT.
